# Prototyping

Creates a neural network which evaluates a time series and produce a set of predicted values for the time series
Predicted values may be used in a policy to make a trade. This policy may be modeled by simple multiple regression or a neural network.

## Data
Test data is taken as most recent to avoid lookahead bias. Train data is split into a validation and training set during fitting.


## TODO
- Convert feature percentages to stdev
- Adding VIX as a signal
- Adding High/Low as signals
- Multiple securities/ aggregate samples
- Policy network
- Regularization (l2)
- Dilated convolution

In [100]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import pandas_datareader as pdr
from datetime import datetime

def from_network(symbol):
    return pdr.get_data_yahoo(symbols=symbol, start=datetime(1900, 1, 1))

def from_file(symbol):
    dataset_path = keras.utils.get_file("{}.csv".format(symbol), "http://localhost:8000/data/daily/{}.csv".format(symbol))
    column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
    return pd.read_csv(dataset_path, 
                              names=column_names, 
                              dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                              header=0,
                              na_values = "?", 
                              comment='\t',
                              sep=",",
                              skipinitialspace=True)

#dataset = raw_dataset.copy()
dataset = from_network('SPY')
dataset = dataset.sort_values(by=['Date'],ascending=False)




In [101]:
print dataset

                  High         Low        Open       Close       Volume  \
Date                                                                      
2019-01-22  265.059998  261.059998  264.820007  262.859985  115355500.0   
2019-01-18  266.980011  263.000000  264.980011  266.459991  127900300.0   
2019-01-17  263.920013  259.959991  260.010010  262.959991   96118400.0   
2019-01-16  261.970001  260.600006  260.829987  260.980011   77636700.0   
2019-01-15  260.700012  257.809998  257.820007  260.350006   85208300.0   
2019-01-14  258.299988  256.410004  256.859985  257.399994   70908200.0   
2019-01-11  259.010010  257.029999  257.679993  258.980011   73858100.0   
2019-01-10  259.160004  255.500000  256.260010  258.880005   96823900.0   
2019-01-09  258.910004  256.190002  257.559998  257.970001   95006600.0   
2019-01-08  257.309998  254.000000  256.820007  256.769989  102512600.0   
2019-01-07  255.949997  251.690002  252.690002  254.380005  103139100.0   
2019-01-04  253.110001  2

In [185]:

dataset_stats = dataset.describe()
dataset_stats = dataset_stats.transpose()
NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 3
NUM_SAMPLES = len(dataset)
NUM_TEST_SAMPLES = int(.25 * NUM_SAMPLES)
print NUM_SAMPLES
# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old

def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation
    return map(lambda arr: map(lambda val: round(val,4),arr), labels)

def convert_to_train(raw_dataset):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(5, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
            dataset['Close'].iloc[i-2] / latest_close, # 3 day trade
            dataset['Close'].iloc[i-3] / latest_close, # 5 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]
converted_feature_set = convert_to_train(dataset)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(converted_feature_set[0][0])]),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(NUM_OUTPUT_NEURONS)
  ])

  model.compile(loss='mse',
                optimizer='sgd',
                metrics=[
                    'mae',
                ])
  return model

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model = build_model()
model.summary()

6542
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_25 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_26 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 51        
Total params: 6,819
Trainable params: 6,819
Non-trainable params: 0
_________________________________________________________________


In [186]:
#len(converted_feature_set[0][0])
print NUM_TEST_SAMPLES
train_data = np.array(converted_feature_set[0][NUM_TEST_SAMPLES:])
train_labels = np.array(converted_feature_set[1][NUM_TEST_SAMPLES:])

test_data = np.array(converted_feature_set[0][:NUM_TEST_SAMPLES])
test_labels = np.array(converted_feature_set[1][:NUM_TEST_SAMPLES])


1635


In [187]:
print train_data
print train_labels

[[ 0.          0.01650096  0.01171268 ... -0.0245304  -0.00957645
  -0.01023941]
 [ 0.         -0.00486862 -0.00471878 ... -0.02651493 -0.02718901
  -0.03954759]
 [ 0.          0.00014911 -0.00864642 ... -0.02221225 -0.03451095
  -0.02116873]
 ...
 [ 0.          0.00280309  0.00630583 ... -0.00840926 -0.00911114
  -0.00490651]
 [ 0.          0.00351259  0.00983838 ... -0.01194772 -0.00773127
   0.00281097]
 [ 0.          0.00634809  0.00775852 ... -0.0112835  -0.0007041
   0.00141044]]
[[0.9976 1.0045 1.0119]
 [1.0168 1.0144 1.0213]
 [0.9952 1.0119 1.0095]
 ...
 [0.9965 0.9944 0.9965]
 [1.0028 0.9993 0.9972]
 [1.0035 1.0063 1.0028]]


In [188]:

history = model.fit(
    train_data, train_labels,
    epochs=70, validation_split = 0.2, verbose=1,
    callbacks=[tensorboard]
)

Train on 3870 samples, validate on 968 samples
Epoch 1/70
3870/3870 [==============================] - 0s 89us/step - loss: 0.3584 - mean_absolute_error: 0.5433 - val_loss: 0.0332 - val_mean_absolute_error: 0.1714
Epoch 2/70
3870/3870 [==============================] - 0s 47us/step - loss: 0.0093 - mean_absolute_error: 0.0720 - val_loss: 6.9278e-04 - val_mean_absolute_error: 0.0216
Epoch 3/70
3870/3870 [==============================] - 0s 42us/step - loss: 0.0018 - mean_absolute_error: 0.0300 - val_loss: 5.4825e-04 - val_mean_absolute_error: 0.0186
Epoch 4/70
3870/3870 [==============================] - 0s 42us/step - loss: 0.0016 - mean_absolute_error: 0.0277 - val_loss: 4.9694e-04 - val_mean_absolute_error: 0.0177
Epoch 5/70
3870/3870 [==============================] - 0s 46us/step - loss: 0.0014 - mean_absolute_error: 0.0262 - val_loss: 4.4048e-04 - val_mean_absolute_error: 0.0167
Epoch 6/70
3870/3870 [==============================] - 0s 42us/step - loss: 0.0012 - mean_absolute_er

3870/3870 [==============================] - 0s 47us/step - loss: 4.5856e-04 - mean_absolute_error: 0.0154 - val_loss: 1.2340e-04 - val_mean_absolute_error: 0.0084
Epoch 48/70
3870/3870 [==============================] - 0s 45us/step - loss: 4.5734e-04 - mean_absolute_error: 0.0153 - val_loss: 1.2413e-04 - val_mean_absolute_error: 0.0084
Epoch 49/70
3870/3870 [==============================] - 0s 44us/step - loss: 4.5594e-04 - mean_absolute_error: 0.0153 - val_loss: 1.2423e-04 - val_mean_absolute_error: 0.0084
Epoch 50/70
3870/3870 [==============================] - 0s 47us/step - loss: 4.5462e-04 - mean_absolute_error: 0.0153 - val_loss: 1.2368e-04 - val_mean_absolute_error: 0.0084
Epoch 51/70
3870/3870 [==============================] - 0s 47us/step - loss: 4.5326e-04 - mean_absolute_error: 0.0153 - val_loss: 1.2323e-04 - val_mean_absolute_error: 0.0084
Epoch 52/70
3870/3870 [==============================] - 0s 47us/step - loss: 4.5195e-04 - mean_absolute_error: 0.0153 - val_loss: 1

In [189]:
outputs = model.predict(test_data)

In [190]:
print test_data[0]
print test_labels[0]
print outputs[0]

[ 0.         -0.00613837 -0.00574985 -0.00221448  0.00244757  0.01173267
  0.01946385  0.05124315  0.02804973  0.02905981  0.03749026  0.03624703
  0.04358975  0.08958818  0.06487955  0.03974357  0.02385392  0.00901318
  0.00792538 -0.01192699 -0.03096349 -0.03131312 -0.02614612 -0.02591303
 -0.02397053 -0.04832946 -0.04992232 -0.08508156 -0.07090132 -0.06441343
 -0.06674434 -0.04273504 -0.03923856 -0.0227273  -0.02960371 -0.02610723
 -0.04545459 -0.06344218 -0.06068374 -0.04972812 -0.05695417 -0.05893556
 -0.07909874 -0.08974362 -0.09172501 -0.06884228 -0.06212129 -0.05629379
 -0.06258748 -0.05139865 -0.04028747 -0.02509709 -0.03080805 -0.04926182
 -0.03076928 -0.06297588 -0.06841498 -0.07323235 -0.07381508 -0.08954941
 -0.08935509 -0.06604507 -0.07206689 -0.05738158]
[1.0115 1.0139 1.0216]
[1.007621  1.0063975 1.0124581]


In [212]:
# primitive policy temporarily in place of a RL policy network

# signal step for our policy network
#signals = map(lambda arr: sum(arr) / len(arr),outputs)
signals = map(lambda arr: sum([arr[0],arr[0],arr[0],arr[1],arr[1],arr[2]]) / 6,outputs)
# signals = map(lambda arr: arr[0],outputs)
trades = map(lambda signal: 1 if round(signal,4) > 1.009 else 0,signals)

In [213]:
# one day return
actual = map(lambda arr: arr[0],test_labels)

df = pd.DataFrame({
    'signal':signals,
    'actual': actual,
    'trade':trades,
})

In [214]:
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)


In [215]:
# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1 or row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df

,actual,signal,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,1.0115,1.008019,0,0,0,0,1,1
1,0.9939,1.009689,1,0,1,0,0,0
2,1.0004,1.015166,1,1,0,0,0,1
3,1.0035,1.023168,1,1,0,0,0,1
4,1.0047,1.020104,1,1,0,0,0,1
5,1.0094,1.017521,1,1,0,0,0,1
6,1.0079,1.017849,1,1,0,0,0,1
7,1.0335,1.013332,1,1,0,0,0,1
8,0.9761,1.011308,1,0,1,0,0,0
9,1.0010,1.014883,1,1,0,0,0,1


In [216]:
print '\nNon-loss events'
print sum(df['success'])
print sum(df['success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nLose trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nMissed opportunities'
print sum(df['avoid_failure'])
print sum(df['avoid_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nBullets dodged'
print sum(df['avoid_success'])
print sum(df['avoid_success']) / (NUM_TEST_SAMPLES * 1.00)


Non-loss events
1621
0.991437308869

Lose trades
14
0.0085626911315

Win trades
32
0.0195718654434

Missed opportunities
857
0.524159021407

Bullets dodged
713
0.436085626911


In [217]:
df1 = dataset.copy().head(NUM_TEST_SAMPLES)
df2 = df.copy()

In [218]:
pd.concat([df1.reset_index(),df2], axis=1).drop(['High','Low','Open','Volume','Adj Close'], axis=1)

,Date,Close,actual,signal,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,2019-01-22,262.859985,1.0115,1.008019,0,0,0,0,1,1
1,2019-01-18,266.459991,0.9939,1.009689,1,0,1,0,0,0
2,2019-01-17,262.959991,1.0004,1.015166,1,1,0,0,0,1
3,2019-01-16,260.980011,1.0035,1.023168,1,1,0,0,0,1
4,2019-01-15,260.350006,1.0047,1.020104,1,1,0,0,0,1
5,2019-01-14,257.399994,1.0094,1.017521,1,1,0,0,0,1
6,2019-01-11,258.980011,1.0079,1.017849,1,1,0,0,0,1
7,2019-01-10,258.880005,1.0335,1.013332,1,1,0,0,0,1
8,2019-01-09,257.970001,0.9761,1.011308,1,0,1,0,0,0
9,2019-01-08,256.769989,1.0010,1.014883,1,1,0,0,0,1
